**Übung Mustererkennung** *WS 2022/23* -- *K. Brandenbusch,  Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung in eingebetteten Systemen*
___
# Aufgabe 4a: Mischverteilungsklassifikator mit klassenunabhängigen Komponentendichten

Statt einer Normalverteilung pro Klasse bietet es sich häufig an, mehrere Verteilungen vorzusehen. Nutzen Sie die in Aufgabe 3 berechneten Cluster und schätzten Sie für jedes Cluster eine Normalverteilung.<br>
Verwenden Sie diese Normalverteilungen als Komponenten eines Mischverteilungsmodells und erstellen Sie mit dessen Hilfe einen Mischverteilungsklassifikator.

In dieser Aufgabe soll zuerst ein Mischverteilungsklassifiktor mit klassenunabhängigen Komponentendichten realisiert werden. Anschließend sollen Sie in Aufgabe 4b einen Mischverteilungsklassifikator mit klassenabhängigen Komponentendichten implementaieren.

Die Klasenbedingten Dichten sollen durch eine Mischverteilung auf der Basis von Normalverteilungsdichten modelliert werden:

$$
    \large p(\underline{c}|\Omega_\kappa) = \sum_{l=1}^{L}p_{\kappa,l}\mathcal{N}(\underline{c}|\underline{\mu}_l,\Sigma_l) \tag*{mit $\large \sum_{l=1}^{L} p_{\kappa,l} = 1, \forall \kappa$.}
$$

$\large \kappa $ ist hier der Index der Klasse und $\large L$ die Anzahl der Komponenten.

---

Zuerst müssen die benötigten Module importiert und der Datensatz geladen werden.

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist

# Uebergeordneten Ordner zum Pfad hinzufuegen, damit das common Package importiert werden kann
if '..' not in sys.path:
    sys.path.append('..')

from common import visualization
from common import log_math
from common.data_provider import DataProvider
from common.vector_quantization import Lloyd

# Trainingsdaten mit Labels laden
train_data_provider = DataProvider(DataProvider.DATA2DROOT_TRAIN)
train_data, train_labels = train_data_provider.get_dataset_and_labels()

---
## Berechnung der Komponentendichten

Verwenden Sie die in Aufgabe 3 implementierte Methode `Lloyd.cluster()` aus dem Modul [`common.vector_quantization`](../common/vector_quantization.py), um ein Kodebuch zu berechnen.<br>
Berechnen Sie anschließend eine Normalverteilung pro Cluster und visualisieren Sie diese.

Überlegen Sie, wie Normalverteilungen definiert sind und wie Sie die nötigen Paramter auf Grundlage der Quantisierung bestimmen können.

Nützliche Funktionen:
 - [np.mean](http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html)
 - [np.cov](http://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html)
 - [visualization.plot_norm_dist_ellipse](../common/visualization.py) (zur Visualisierung)


---
## Zuordnung der Beispiele zu den Komponentendichten

Bisher erfolgte die Zuordnung der Daten zu den Kodewörtern auf Basis ihrer Distanz zu den Kodewörtern.<br>
Ordnen Sie nun die Beispiele der Normalverteilung zu, welche diese mit größter Likelihood generiert und visualisieren Sie die Zuordnung.<br>

**Optional:** Visualisieren Sie zusätzlich, bei welchen Datenpunkten sich die Zuordnung geändert hat.

---
## Gewichtung der Komponentendichten

Damit aus den Komponentendichten eine gültige Mischverteilung erstellt werden kann, müssen die Komponentendichten gewichtet werden mit $\large \sum_{l=1}^{L} p_{\kappa,l} = 1, \forall \kappa $.<br>
Berechnen Sie nun $\large p_{\kappa,l} $ für alle Klassen und Komponenten.<br>
Visualisieren Sie anschließend in jeweils einem Plot die Daten aus der Stichprobe einer Klasse und die Gewichtung $\large p_{\kappa,l} $ aller Komponenten für die jeweilige Klasse.

In derselben Schleife können Sie bereits die a-priori Wahrscheinlichkeit $\large p_\kappa $ der drei Klassen berechnen.

---
## Anwendung der Entscheidungsregel

Basierend auf den zuvor berechneten Parametern können Sie nun die klassenbedingten Dichten $\large p(\underline{c}|\Omega_\kappa) $ für gegebene Merkmalsvektoren $\large \underline{c} $ auswerten.<br>
Laden Sie den Testdatensatz und klassifizieren Sie alle Beispiele. Die Entscheidungsregel lautet wie in Aufgabe 2:

$$
    \large \lambda = \underset{\kappa \in \{1,\dots,K\}}{\text{argmax}} p_\kappa p(\underline{c}|\Omega_\kappa).
$$

---
## Evaluation des Klassifikators

Bewerten Sie die Klassifikationsergebnisse mit dem `ClassificationEvaluator` aus dem Modul [`common.classification`](../common/classification.py).

**Optional:** Plotten Sie die Trainings- und die Testdaten und kennzeichnen Sie farblich die zugehörige bzw. zugewiesene Klasse.

---
## Implementierung des Klassifikators

Implementieren Sie basierend auf dem bishererigen Code die Klasse `MDClassifierClassIndep` im Modul [`common.classification`](../common/classification.py):

Überlegen Sie, welche Parameter für einen Mischverteilungsklassifikator (mit klassenunabhängigen Komponentendichten) für die Klassifikation benötigt werden.<br>
Legen Sie Variablen für diese Parameter an und initialisieren Sie diese in der Methode `MDClassifierClassIndep.__init__()`.<br>
In der Methode `MDClassifierClassIndep.estimate()` sollen diese Parameter basierend auf den übergebenen Trainingsdaten geschätzt werden.<br>
Die Methode `MDClassifierClassIndep.classify()` soll Klassenlabels (nicht die IDs der Klassen!) für die übergebenen Merkmalsvektoren vorhersagen.

Instanzieren Sie den von Ihnen implementierten Klassifikator und trainieren Sie diesen. Testen Sie diesen anschließend auf den Testdaten.<br>
Bewerten Sie die Klassifikation und vergleichen diese mit den zuvor erhaltenen Ergebnissen.